# Project problem statement

Compare the Spectral Element Method (SEM) formulation used in Lee & Jang (2012) for 1‑D viscoelastic blood flow with a small set of alternative numerical approaches and linear‑algebra diagnostics. The comparison emphasizes

- convergence and accuracy,
- stability and timestep restrictions (CFL behavior),
- stiffness handling and implicit solvers (Newton),
- behavior and robustness of shooting methods for boundary‑value problems, and
- how linear algebra diagnostics (norms, condition numbers, QR, SVD, eigenvalues) explain solver performance.

## Target PDE (reduced Lee & Jang form)

$$
M \ddot{u} + C \dot{u} + G \, \partial_x \dot{u} + D \, \partial_{xx} u + A \, \partial_x u + K u = f_e + f_{NL}(u),
$$

with

$$
u = \begin{bmatrix} p \\ q \end{bmatrix}
$$

(pressure & flow correction). Use the same physical parameters and boundary conditions as Lee & Jang for the artery test case.

## Project goal (deliverable in 8-10 days)

- Implement a minimal SEM (single‑element linear + simplified AFT nonlinear iteration) to reproduce Lee & Jang behavior.
- Implement a time‑domain FEM (P1 elements) with Crank–Nicolson + Newton (implicit).
- Implement a simple shooting method (IVP rootfinding) for a related BVP or steady problem to compare robustness.
- Use SVD / condition number / QR / eigenanalysis to explain differences in solver behavior (convergence, Newton iterations, CFL limits).
- Run a manufactured‑solution convergence study and the Lee & Jang example (coarse resolutions). Deliver error vs resolution, stability/CFL test for FD/explicit time stepping, Newton iteration counts, and linear algebra diagnostics (SVD, cond).

## Why these choices (short justification)

- **SEM (Lee & Jang):** spectral accuracy for wave propagation; AFT handles nonlinearities in frequency domain — acts as a high‑accuracy reference.
- **FEM implicit (CN+Newton):** robust, handles stiffness & nonlinearities naturally; standard benchmark.
- **Shooting (IVP rootsolve):** compact alternative for 2nd‑order BVPs; highlights sensitivity and stiffness issues.
- **SVD / QR / cond / eigen:** diagnose ill‑conditioning and explain solver performance; guide preconditioning.

## Key references

- Lee & Jang (2012), *Spectral element modeling and analysis of the blood flows in viscoelastic vessels*.
- U. Lee, *Spectral Element Method in Structural Dynamics*.
- Hairer, Nørsett & Wanner, *Solving Ordinary Differential Equations*.
- Trefethen & Bau, *Numerical Linear Algebra*.

# Concrete approach (minimal 10‑day plan)

Use Python (NumPy / SciPy / matplotlib). Keep meshes small for fast runs. Use a manufactured solution for convergence and the paper's artery case for comparison.

---

### Day 0 (prep)

- Extract parameters from the paper,
- Set up repo skeleton and helper functions (`mesh.py`, `assemblers.py`, `sem.py`, `fem.py`, `fd.py`, `io_plot.py`),
- Choose time window $T$, coarse grids (e.g., $N_x=25,50$), and spectral time sample $N_t$ (power of two).

---

### Day 1-2 — FEM CN linear + manufactured tests

- Assemble P1 FEM matrices for the linearized problem: $M$, $K$ (plus any constant coefficient matrices).

- Implement Crank–Nicolson (CN) time step for linear PDE:

  $$
  \left(M + \frac{\Delta t}{2} K \right) u^{n+1} = \left(M - \frac{\Delta t}{2} K \right) u^n + \Delta t \, f^{n+\frac{1}{2}}.
  $$

- Implement manufactured linear solution

  $$
  u_{ex}(x,t) = \sin\left(\frac{\pi x}{L}\right) \cos(\omega t)
  $$

  and verify second‑order convergence in space/time (two resolutions).

- **Output:** CN FEM linear solver verified.

---

### Day 3-4 — FEM nonlinear + Newton; shooting

- Add nonlinear residual $f_{NL}(u)$ (use Lee & Jang expressions or a simplified quadratic nonlinearity).

- Implement Newton per CN step:
  - residual $R(u)$,
  - Jacobian $J(u)$ (finite‑difference Jacobian allowed),
  - solve $J \delta = -R$ with `scipy.sparse.linalg.spsolve`; record Newton iteration counts.

- Implement a basic shooting method for a related 2‑pt nonlinear BVP (reduce to IVP with unknown slope; use `solve_ivp` + `scipy.optimize.root_scalar`). Test sensitivity.

- **Output:** FEM+Newton working; shooting robustness demo.

---

### Day 5-6 — SEM linear single‑element + frequency response

- Implement SEM linear builder for a single element:
  - for each frequency $\omega$ (from spectral grid) compute dispersion polynomial roots $k_1,k_2$ (use `numpy.roots`),
  - assemble element dynamic stiffness $S(\omega)$ (use paper formulas; if algebraic integrals are long, compute with Gaussian quadrature),
  - global $S_g(\omega)$ (1 element $\Rightarrow S_g = S$),
  - solve spectral system across harmonics, inverse FFT to get time trace.

- Compare SEM linear time trace to FEM linear CN result (same coarse discretization); compute $L_2$ difference at sensor(s).

- **Output:** SEM linear validated.

---

### Day 7-8 — SEM AFT simplified + FD RK4 CFL test

- Implement simplified AFT loop (limit iterations to 3–6):
  - initialize spectral solution from linear SEM,
  - IFFT → compute $f_{NL}(t)$ → FFT → for each $\omega$ solve $S_g(\omega), d_p = \text{RHS}(\omega)$ (cache LU per $\omega$), iterate.

- Implement FD 2nd‑order central + RK4 explicit integrator; perform CFL sweep (increase $\Delta t$ until blowup) and report max stable $\Delta t$.

- Implement FD implicit CN using the FEM Newton routine (reuse assembly).

- **Output:** SEM AFT run + FD explicit stability data.

---

### Day 9-10 — Tests, diagnostics, plots, short report

- **Run tests:**
  - manufactured solution at two resolutions for FEM & FD; vary SEM spectral resolution a bit; collect $L_2$ errors,
  - Lee & Jang artery example: SEM (1 element), FEM coarse, FD coarse; collect sensor time traces.

- **Compute diagnostics:**
  - SVD / singular values and condition number on Jacobian (FEM coarse) and one representative $S(\omega)$ (SEM) using `numpy.linalg.svd` and `numpy.linalg.cond`,
  - QR on small matrices where projection stability matters,
  - eigenvalues of linearized operator (estimate via `scipy.sparse.linalg.eigs`) and map $z = \lambda \Delta t$ if desired.

- **Produce plots:**
  - Manufactured test: error vs resolution (log–log).
  - Time series $p(t)$ at sensor(s) for artery test (overlaid methods).
  - FD explicit CFL plot (max stable $\Delta t$ vs $\Delta x$).
  - Newton iteration counts & runtime bar chart.
  - Singular value spectra (Jacobian and $S(\omega)$).

- Write a 2–3 page summary: accuracy, stability, stiffness handling, and linear algebra insights.

# Minimal numerical methods & implementation notes

---

### FEM (CN + Newton)

- Discretize space with P1 elements; assemble $M$, $K$, and other matrices $C, D, A, G$ as needed.
- Time step with Crank–Nicolson (A‑stable).
- Nonlinear solve via Newton each CN step. Finite‑difference Jacobian allowed.
- Solver: `scipy.sparse.linalg.spsolve`.

---

### SEM (single‑element linear + AFT)

- Use dynamic shape functions and derive $S(\omega)$ for a single element (follow Lee & Jang).
- Frequency grid from $N_t$ points (power of two), use `numpy.fft.rfft` / `irfft`.
- AFT: limited iterations; cache LU factorizations for each frequency.

---

### Shooting (toy BVP)

- Reduce a 2‑pt nonlinear BVP to initial slope rootfinding.
- Use `solve_ivp` and `scipy.optimize.root_scalar` or `fsolve`.
- Demonstrate basin sensitivity and stiffness effects.

---

### FD (central 2 + RK4 & CN)

- Spatial stencils: centered 2nd order for $\partial_x$ and $\partial_{xx}$.
- RK4 explicit: test CFL limits.
- CN implicit: reuse FEM Newton solver.

---

### Linear algebra diagnostics

- SVD: `U, s, V = np.linalg.svd(A, full_matrices=False)` (inspect $s$).
- Condition number: `np.linalg.cond(A)`.
- QR: `Q, R = np.linalg.qr(A)` for orthonormalization.

---

### Key implementation shortcuts

- Use small DOF coarse grids ($N_x=25\text{–}50$) for speed.
- Use finite‑difference Jacobian if analytic Jacobian is too time‑consuming; validate via directional derivative check.
- For SEM integrals with long closed forms, use Gaussian quadrature with few points.
- Limit AFT iterations (3–6) for prototype; report convergence behavior.